## adjust countries to be the same information

In [42]:
%pip install geopy translate joblib

Note: you may need to restart the kernel to use updated packages.


### Reading files automatically

In [43]:
import os
import pandas as pd

def _adjust_name(name):
    if not('df_' in name):
        return 'df_' + name
    return name
def _get_files_from(base_path = '..\\data\\processed', extension='csv'):
    return [(_adjust_name(name.split('.')[0].lower()), str(os.path.join(root, name))) for (root, _, files) in os.walk(base_path, topdown=False) for name in files if (extension in name)]

def read_files(_from='..\\data\\processed', extension='csv'):
    vars = []
    for item in _get_files_from(_from, extension=extension):
        exec(f'vars.append("{item[0]}")')
        exec(f"{item[0]} = pd.read_csv(r'{item[1]}', sep=';', decimal=',')")
    return vars

# _get_files_from()

## Load data

In [59]:
import os
import pandas as pd
import numpy as np
import joblib
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots

base_path = '..\\data\\processed'

df_noaa_global = pd.read_csv(r'..\\data\\processed\\noaa_global\\noaa_global_final.csv', sep=';', decimal=',')
df_vinhos = pd.read_csv(r'..\\data\\processed\\tech_challenge\\df_vinhos.csv', sep=';', decimal=',')
df_temperature_change_data = pd.read_csv(r'..\\data\\processed\\temp_change\\temperature_change_Data.csv', sep=';', decimal=',')
df_wbpy = pd.read_csv(r'..\\data\\processed\\wbpy\\wbpy.csv', sep=';', decimal=',')
df_clima_rs =  pd.read_csv(r'..\\data\\processed\\inmet\\rs_final.csv', sep=';', decimal=',')

## Dados climáticos RS

In [60]:
stat='TAVG'
thresold_filter = 6
dict_y_transformation={
            'PRCP':('value_median','Mediana da Precipitação')
            , 'TAVG':('value_mean','Média da Temperatura')
            , 'TMAX':('value_max','Máxima da Temperatura')
            , 'TMIN':('value_min','Mínima da Temperatura')
        }

In [61]:
# Primeiro, recuperamos a variavel a ser analisada
df_precp_global = df_noaa_global.loc[df_noaa_global['stat']==stat]
# preencher null
df_precp_global[dict_y_transformation[stat][0]] = df_noaa_global[dict_y_transformation[stat][0]].fillna(df_noaa_global.groupby(['year','stat','country_code'])[dict_y_transformation[stat][0]].transform('mean'))
# vamos remover paises com menos de 3 pontos de dados
df_precp_global = df_precp_global[['year','country_code',dict_y_transformation[stat][0]]]
df_precp_global = df_precp_global.loc[df_precp_global.year >= min(df_clima_rs.year)]
df_precp_global = df_precp_global.loc[df_precp_global.year <= max(df_clima_rs.year)]
df_clima_rs = df_clima_rs.rename(columns={stat:dict_y_transformation[stat][0]})
df_clima_rs['country_code'] = 'BR-RS'
# e criamos um dataframe com todos os dados
df_full = pd.concat([df_precp_global.loc[~df_precp_global['country_code'].isin(['BR'])]])

C:\Users\Pehls\AppData\Local\Temp\ipykernel_19448\2000450634.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [66]:
df_precp_global.dropna().country_code.value_counts().reset_index().query('count > 5')

,country_code,count
0,AF,13
1,HU,13
2,IR,13
3,AR,13
4,IT,13
5,JP,13
6,LU,13
7,LV,13
8,ME,13
9,NL,13


In [54]:
def _mape(real, comparado):
    return list(abs(np.array(real) - comparado)/comparado)

_max_precip = max(df_clima_rs[dict_y_transformation[stat][0]])
_min_precip = min(df_clima_rs[dict_y_transformation[stat][0]])
_mean_precip = np.mean(df_clima_rs[dict_y_transformation[stat][0]])

# para cada bloco, 
# 1 - calculamos o ape
df_full['dif_perc_max'] = _mape(df_full[dict_y_transformation[stat][0]], _max_precip)
# 2 - ranqueamos, para cada ano
df_full['rank_per_max_year'] = df_full.groupby('year')['dif_perc_max'].rank(method="dense")
# 3 - somamos os ranks, a menor soma vai ser o pais mais bem colocado
final_rank_max = df_full.groupby(['country_code']).agg({'rank_per_max_year':'sum'}).reset_index()
# 4 -e ranqueamos novamente para filtrar mais pra frente
final_rank_max['final_rank_max'] = final_rank_max['rank_per_max_year'].rank(method="dense")
df_full = df_full.drop(columns=['rank_per_max_year']).merge(final_rank_max.drop(columns=['rank_per_max_year']))

df_full['dif_perc_min'] = _mape(df_full[dict_y_transformation[stat][0]], _min_precip)
df_full['rank_per_min_year'] = df_full.groupby('year')['dif_perc_min'].rank(method="dense")
final_rank_max = df_full.groupby(['country_code']).agg({'rank_per_min_year':'sum'}).reset_index()
final_rank_max['final_rank_min'] = final_rank_max['rank_per_min_year'].rank(method="dense")
df_full = df_full.drop(columns=['rank_per_min_year']).merge(final_rank_max.drop(columns=['rank_per_min_year']))

df_full['dif_perc_mean'] = _mape(df_full[dict_y_transformation[stat][0]], _mean_precip)
df_full['rank_per_mean_year'] = df_full.groupby('year')['dif_perc_mean'].rank(method="dense")
final_rank_max = df_full.groupby(['country_code']).agg({'rank_per_mean_year':'sum'}).reset_index()
final_rank_max['final_rank_mean'] = final_rank_max['rank_per_mean_year'].rank(method="dense")
df_full = df_full.drop(columns=['rank_per_mean_year']).merge(final_rank_max.drop(columns=['rank_per_mean_year']))

# finalmente, filtramos os 6 menores (o 1 sera sempre o proprio, esperamos)
# df_full = df_full.query(f'(final_rank_max <= {thresold_filter} or final_rank_min <= {thresold_filter}) or final_rank_mean <= {thresold_filter}')


In [55]:
df_full.query(f'(final_rank_max <= {thresold_filter} or final_rank_min <= {thresold_filter}) or final_rank_mean <= {thresold_filter}')


,year,country_code,value_mean,dif_perc_max,final_rank_max,dif_perc_min,final_rank_min,dif_perc_mean,final_rank_mean
0,2007,AE,NaN,NaN,1.0,NaN,1.0,NaN,1.0
1,2008,AE,NaN,NaN,1.0,NaN,1.0,NaN,1.0
2,2009,AE,NaN,NaN,1.0,NaN,1.0,NaN,1.0
3,2010,AE,NaN,NaN,1.0,NaN,1.0,NaN,1.0
4,2011,AE,NaN,NaN,1.0,NaN,1.0,NaN,1.0
...,...,...,...,...,...,...,...,...,...
1231,2015,IQ,NaN,NaN,1.0,NaN,1.0,NaN,1.0
1232,2016,IQ,NaN,NaN,1.0,NaN,1.0,NaN,1.0
1233,2017,IQ,NaN,NaN,1.0,NaN,1.0,NaN,1.0
1234,2018,IQ,NaN,NaN,1.0,NaN,1.0,NaN,1.0


In [49]:

fig = go.Figure()

fig = px.violin(
        df_full, 
        x='country_code', y='value_median', color='country_code'
        , hover_data=['year']
        , box=True
        , points='all'
    )


fig.show()

ValueError: Value of 'y' is not the name of a column in 'data_frame'. Expected one of ['year', 'country_code', 'value_mean', 'dif_perc_max', 'final_rank_max', 'dif_perc_min', 'final_rank_min', 'dif_perc_mean', 'final_rank_mean'] but received: value_median

In [ ]:
df_full.sort_values(['year','value_median'])['country_code'].value_counts()

country_code
BR-RS    16
IE       13
TG       13
BO       13
PY       13
VU       13
CR       13
TH       13
ID       13
LV       12
GQ       10
BB        3
NI        2
Name: count, dtype: int64